In [1]:
# some standard packages
import numpy as np
import pandas as pd
import pickle
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

# modelling packages
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten 
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau 
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import model_from_json
from tensorflow.keras.models import load_model
from tensorflow.keras.optimizers import Adam


# Model Evaluation
from sklearn.metrics import classification_report, confusion_matrix


In [2]:
categories = ['hogweed', 'blueweed', 'gorse']

In [3]:
# Use the Keras ImageDataGenerator for memory efficiency and preprocessing ease
# This process replaces the method of obtaining our data via DataLoader.ipynb
train_datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    rescale=1./255,
    fill_mode='nearest',
    validation_split=0.2,
)

In [4]:
test_datagen = ImageDataGenerator(rescale = 1./255)

In [5]:
batch_size = 32

In [6]:
train_generator = train_datagen.flow_from_directory(
                                                    'data/final_BC_images/train',
                                                    target_size=(32, 32),
                                                    color_mode='rgb',
                                                    batch_size=batch_size,
                                                    class_mode='categorical',
                                                    shuffle=True,
                                                    subset='training')

Found 4862 images belonging to 3 classes.


In [7]:
validation_generator = train_datagen.flow_from_directory(
                                                        'data/final_BC_images/train',
                                                        target_size=(32, 32),
                                                        color_mode='rgb',
                                                        batch_size=batch_size,
                                                        class_mode='categorical',
                                                        shuffle=False,
                                                        subset='validation'
                                                        )

Found 1215 images belonging to 3 classes.


In [8]:
test_generator = test_datagen.flow_from_directory(
                                                  'data/final_BC_images/test',
                                                  target_size=(32, 32),
                                                  color_mode='rgb',
                                                  batch_size=batch_size,
                                                  class_mode='categorical',
                                                  shuffle=False)

Found 1632 images belonging to 3 classes.


In [9]:
# Saving the number of stepsizes for the training, validation and test sets 
train_stepsize = train_generator.samples//train_generator.batch_size 

valid_stepsize = validation_generator.samples//validation_generator.batch_size 

test_stepsize = test_generator.samples//test_generator.batch_size 

# Sanity check 
print(f'Training step size = {train_stepsize} \nValidation step size = {valid_stepsize} \nTest step size = {test_stepsize}')

Training step size = 151 
Validation step size = 37 
Test step size = 51


In [10]:
# Building the Model
model = Sequential()

# 3 convolutional layers
model.add(Conv2D(32, (3,3), input_shape = (32,32,3)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(128, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(512, (3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

# 3 hidden layers
model.add(Flatten())
model.add(Dense(256))
model.add(Activation('relu'))

model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dropout(0.25))

model.add(Dense(128))
model.add(Activation('relu'))

# # The output layer with 9 neurons for 9 classes
model.add(Dense(3))
model.add(Activation('softmax'))


In [11]:
# Initiate early stop based on validation accuracy
ES = EarlyStopping(monitor='val_acc', patience=5, mode='auto', min_delta=0.0001, verbose=1)

In [12]:
# Istantiating Adam optimizer with a learning rate of 0.0001 and saving to variable 'optim'
optim = Adam(lr=0.001)

# Compiling the CNN model 
model.compile(optimizer=optim, loss='categorical_crossentropy', metrics=['acc'])

# Summary 
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 30, 30, 32)        896       
_________________________________________________________________
activation (Activation)      (None, 30, 30, 32)        0         
_________________________________________________________________
batch_normalization (BatchNo (None, 30, 30, 32)        128       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 13, 13, 128)       36992     
_________________________________________________________________
activation_1 (Activation)    (None, 13, 13, 128)       0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 6, 6, 128)         0

In [13]:
# Fitting the model to the training data
history = model.fit_generator(generator=train_generator,
                                steps_per_epoch=train_stepsize,
                                epochs=50,
                                validation_data=validation_generator,
                                validation_steps=valid_stepsize,
                                callbacks=[ES])

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/50
151/151 [==============================] - 61s 404ms/step - loss: 0.6806 - acc: 0.7076 - val_loss: 0.9785 - val_acc: 0.4890
Epoch 2/50
151/151 [==============================] - 58s 382ms/step - loss: 0.4913 - acc: 0.8124 - val_loss: 0.8301 - val_acc: 0.5760
Epoch 3/50
151/151 [==============================] - 59s 392ms/step - loss: 0.4021 - acc: 0.8578 - val_loss: 0.5431 - val_acc: 0.7821
Epoch 4/50
151/151 [==============================] - 58s 386ms/step - loss: 0.3815 - acc: 0.8540 - val_loss: 0.6574 - val_acc: 0.6917
Epoch 5/50
151/151 [==============================] - 59s 388ms/step - loss: 0.3604 - acc: 0.8654 - val_loss: 0.4090 - val_acc: 0.8530
Epoch 6/50
151/151 [==============================] - 57s 380ms/step - loss: 0.3465 - acc: 0.8752 - val_loss: 0.8646 - val_acc: 0.7171
Epoch 7/50
151/151 [==============================] - 59s 388ms/step - loss: 0.3411 - acc: 0.8720 - val_loss: 0.40

The results look promisinng for some epochs, but the validation loss and accuarcy are hopping around way too much. The model seems to be unstable.

In [ ]:
# Getting bestmodel's predictions (as probabilities) on the test set 
test_probas = model.predict_generator(test_generator, steps=test_stepsize)

# Setting the model's class prediction as the class that received the highest probability for each image
test_predictions = test_probas.argmax(axis=1)

In [ ]:
len(test_probas)

In [ ]:
# Getting the true class labels for the test set
test_true = test_generator.classes

# Sanity check 
len(test_true)

In [ ]:
len(test_predictions)

In [ ]:
# Displaying the classification report for the test set
print('Classification Report\n \n', classification_report(test_true, test_predictions, target_names=categories))

In [ ]:
# Get a confusion matrix 
test_matrix = pd.DataFrame(confusion_matrix(test_true, test_predictions), 
                           columns=['Predicted ' + cat_name for cat_name in categories], 
                           index=['True ' + cat_name for cat_name in categories])

# Plotting as a heatmap 
plt.figure()
sns.heatmap(test_matrix, cmap='Blues', annot=True, fmt='g')
plt.title('Normalized Confusion Matrix: Test Data')
plt.show()

In [ ]:
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, stratify = y) 

In [ ]:
# # Train the model
# history = model.fit_generator(generator=train_generator,
#                     steps_per_epoch=(11209) // batch_size,
#                     epochs=50, 
#                     validation_data=validation_generator,
#                     validation_steps=(2798) // batch_size,
#                     callbacks=[
#                         EarlyStopping(patience=3, restore_best_weights=True),
#                         ReduceLROnPlateau(patience=2)],
#                     verbose=1)